In [34]:
from __future__ import division

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import concat, col, lit
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from IPython.display import clear_output
import numpy as np
from pyspark.ml.linalg import SparseVector
from operator import attrgetter
from scipy.sparse import vstack
from scipy.sparse import csr_matrix

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import concat, col, lit
import pandas as pd

from sklearn.feature_selection import chi2,SelectKBest
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import numpy as np
from pyspark.ml.linalg import SparseVector
from operator import attrgetter
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from pyspark.ml.feature import CountVectorizer
from pylab import barh,plot,yticks,show,grid,xlabel,figure
import nltk

def cleantext( raw_data):
    letters_only = re.sub("[^a-zA-Z0-9]", " ", raw_data) 
    words = word_tokenize(letters_only.lower())                         
    meaningful_words = [ WordNetLemmatizer().lemmatize(w,'v') for w in words if not w in stops]
    meaningful_words = [w for w in meaningful_words if (len(w) >2) and (not w in white) ]
    return (' ').join(meaningful_words)

def feature_select(x):
    if ' 'in x[0]:
        temp=nltk.pos_tag(nltk.word_tokenize(x[0]))
        temp=zip(*temp)
        term=[('JJ','NN'),('NN','JJ'),('RB','VB'),('RB','NN'),('RB','JJ'),('NN','NN'),('RB','RB')]
        return( temp[1] in term)

def as_matrix(vec):
    data, indices = vec.values, vec.indices
    shape = 1, vec.size
    return csr_matrix((data, indices, np.array([0, vec.values.size])), shape)


productlist=pd.read_excel('productinfo.xlsx')
bwlist=pd.read_excel('whitelist.xlsx')
white=bwlist['white'].tolist()
black=set(bwlist['black'].tolist())
productname=pd.read_excel('productname.xlsx')
stemmer = PorterStemmer()
stops = set(stopwords.words("english"))
stops=stops|black

amazon = sqlContext.read.json("Electronics_5.json")
amazon1=amazon.withColumn("output",(amazon['overall']>3).cast('int').cast('int'))
amazon1=amazon1.select('*',concat(col("summary"), lit(" "), col("reviewText")).alias("reviewsum"))
#amazon1=amazon1.filter(amazon1.output>-1)
amazon1=amazon1.drop('reviewText')
clean = udf(lambda s: cleantext(s), StringType())
amazon1= amazon1.withColumn('reviewsum', clean(amazon1['reviewsum']))



gram=list()
onew=list()
tword=list()
ti1=list()
ti2=list()
ti3=list()
cou=list()
import timeit
from itertools import combinations
from sklearn.feature_selection import chi2,SelectKBest
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pyspark.ml.feature import CountVectorizer # Import the stop word list
#from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import numpy as np
from pyspark.ml.linalg import SparseVector
from operator import attrgetter
from scipy.sparse import vstack
from scipy.sparse import csr_matrix

from pylab import barh,plot,yticks,show,grid,xlabel,figure
import nltk

for x in productlist['asin'][::-1][-20:]:
    print(x)
    amazontemp=amazon1.filter(amazon1.asin==x)
    start = timeit.default_timer()
    tokenizer = Tokenizer(inputCol="reviewsum", outputCol="words")
    rating_time = tokenizer.transform(amazontemp)
    
    cv = CountVectorizer(inputCol="words", outputCol="vectors", minDF=3.0)
    model = cv.fit(rating_time)
    #result = model.transform(rating_time)
    vocab_term=model.vocabulary
    onew.append(len(vocab_term))
    
    
    stop1 = timeit.default_timer()
    
    from pyspark.ml.feature import NGram
    ngram = NGram(n=2,inputCol="words", outputCol="ngrams")
    ngramDataFrame = ngram.transform(rating_time)
    rating_time=ngramDataFrame
    
    #form term frequency table
    #cv = CountVectorizer(inputCol="ngrams", outputCol="vectors", minDF=3.0)
    #model = cv.fit(rating_time)
    #result = model.transform(rating_time)
    #vocab_term=model.vocabulary
    #gram.append(len(vocab_term))
    
    twcom=[]
    def twoword(doc):
        term=doc.split()
        temp=[]
        #temp += ["vvv".join(a) for a in combinations(term[0:(0+4)], 2)]
        for i in (range(1,len(term)-4)):
            for k in range(0,3):
                temp += [term[i+k]+'vvv'+term[i+k+1]]
        return (" ".join(temp))

    tw = udf(lambda s: twoword(s), StringType())
    rating_time=rating_time.withColumn('reviewsum1', tw(ngramDataFrame['reviewsum']))
    from pyspark.ml.feature import HashingTF, IDF, Tokenizer
    tokenizer = Tokenizer(inputCol="reviewsum1", outputCol="words1")
    wordsData = tokenizer.transform(rating_time)
    cv = CountVectorizer(inputCol="words1", outputCol="vectors", minDF=3.0)
    model = cv.fit(wordsData)
    #result = model.transform(wordsData)
    vocab_term=model.vocabulary
    tword.append(len(vocab_term))
    
    stop2 = timeit.default_timer()
    
    twcom=[]
    def twoword(doc):
        term=doc.split()
        temp=[]
        temp += ["vvv".join(a) for a in combinations(term[0:(0+4)], 2)]
        for i in (range(1,len(term)-4)):
            for k in range(0,3):
                temp += [term[i+k]+'vvv'+term[i+4]]
        return (" ".join(temp))

    tw = udf(lambda s: twoword(s), StringType())
    rating_time=rating_time.withColumn('reviewsum1', tw(ngramDataFrame['reviewsum']))
    from pyspark.ml.feature import HashingTF, IDF, Tokenizer
    tokenizer = Tokenizer(inputCol="reviewsum1", outputCol="words1")
    wordsData = tokenizer.transform(rating_time)
    cv = CountVectorizer(inputCol="words1", outputCol="vectors", minDF=3.0)
    model = cv.fit(wordsData)
    #result = model.transform(wordsData)
    vocab_term=model.vocabulary
    tword.append(len(vocab_term))
    
    stop3 = timeit.default_timer()
    
    ti1.append(stop1-start)
    ti2.append(stop2-start)
    ti3.append(stop3-stop2)

cou=productlist['count(summary)'][::-1][-20:]
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

line1, = plt.plot(cou,ti1, marker='o', label='single word')
line2, = plt.plot(cou,ti2, marker='o', label='2 grams')
line3, = plt.plot(cou,ti3, marker='o', label='2 words combination')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

ti22=list()
for x in range(20):
    ti22.append(ti2[x]-ti1[x])

cou=productlist['count(summary)'][::-1][-20:]
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(cou,ti1, marker='o', label='single word')
line2, = plt.plot(cou,ti22, marker='o', label='2 grams')
line3, = plt.plot(cou,ti3, marker='o', label='2 words combination')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

cou=productlist['count(summary)'][::-1][-20:]
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

line1, = plt.plot(cou,onew, marker='o', label='single word')
line2, = plt.plot(cou,gram, marker='o', label='2 grams')
line3, = plt.plot(cou,tword, marker='o', label='2 words combination')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

